In [1]:
import shapely
from arcgis.gis import GIS, Item
from arcgis.geometry import Geometry, Point, Polyline
from shapely import ops

from geometry_monkeypatch import *

In [2]:
item_id = 'cddda6482e4540158aa705675b33371c'
user_id = 'joel_mccune'
feature_id = 603

In [3]:
gis = GIS(username=user_id)
gis

In [4]:
layer = Item(gis, item_id).layers[0]
layer

<FeatureLayer url:"http://services5.arcgis.com/12oODIpfxlRR11MF/arcgis/rest/services/hydroline/FeatureServer/0">

In [5]:
fs = layer.query(object_ids=603)
sdf = fs.df

/opt/conda/lib/python3.6/site-packages/arcgis/features/feature.py:613: UserWarning: The SpatialDataFrame has been deprecated. `df` property be modified to return the Spatially Enabled DataFrame as v2.0. This property should not be used. Please use `as_df` instead.
  warnings.warn(("The SpatialDataFrame has been deprecated. "
/opt/conda/lib/python3.6/site-packages/arcgis/features/_data/geodataset/geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


In [6]:
hydroline = sdf.geometry[0]  # first geometry
path = hydroline.paths[0]  # path 
last_idx = len(path) - 1

putin = Geometry(x=path[0][0], y=path[0][1], spatialReference={'wkid': 4326})
takeout = Geometry(x=path[last_idx][0], y=path[last_idx][1], spatialReference={'wkid': 4326})

In [7]:
# arbitrally collect a single point as an Esri Point as a starting point for prototyping - the geomeric centroid
hydropoint = Geometry(x=-72.8, y=44.8, spatialReference={'wkid': 4326})

In [8]:
snapped = hydropoint.snap_to_line(hydroline)
print(hydropoint)
print(snapped)

{'x': -72.8, 'y': 44.8, 'spatialReference': {'wkid': 4326}}
{'x': -72.745189798, 'y': 44.743769931, 'spatialReference': {'wkid': 4326}}


In [9]:
hydroline.trim_at_point(snapped)

{'paths': [[(-72.741389671, 44.7436323210001),
   (-72.7416639309999, 44.7436537310001),
   (-72.741997198, 44.7436977310001),
   (-72.742361331, 44.7437663310001),
   (-72.7428287979999, 44.743853131),
   (-72.743392531, 44.743985997),
   (-72.743619931, 44.744016931),
   (-72.743804331, 44.744035797),
   (-72.743978731, 44.744038597),
   (-72.7440923309999, 44.7440353310001),
   (-72.7444153979999, 44.744008997),
   (-72.744647731, 44.743987997),
   (-72.744749398, 44.743964197),
   (-72.744786531, 44.743955397),
   (-72.745025998, 44.7438593970001),
   (-72.745189798, 44.743769931)]],
 'spatialReference': {'wkid': 4269, 'latestWkid': 4269}}

In [12]:
from reach_tools import *

In [20]:
waters = WATERS()

In [21]:
epa_putin = waters.get_epa_snap_point(putin.x, putin.y)
print(putin)
print(epa_putin)

{'x': -72.741389671, 'y': 44.7436323210001, 'spatialReference': {'wkid': 4326}}
{'geometry': {'x': -72.7413852696573, 'y': 44.7435467332978, 'spatialReference': {'wkid': 4269}}, 'measure': 8.99219, 'id': 4586212}


In [23]:
epa_putin = waters.get_epa_snap_point(putin.x, putin.y)
trace_response = waters._get_epa_upstreamdownstream_response(epa_putin)
trace_geom = waters._epa_trace_resp_to_esri_geom(trace_response)
trace_geom

{'paths': [[(-72.7413852715841, 44.743546734191),
   (-72.7419019307288, 44.7435333301),
   (-72.7424789978068, 44.7433955305802),
   (-72.7440173304348, 44.7427541304996),
   (-72.74481839795, 44.7423191302246),
   (-72.7445611972408, 44.7419765307932),
   (-72.744592731069, 44.7416793309373),
   (-72.7441423973531, 44.7410625974618),
   (-72.7441415978558, 44.7406283975834),
   (-72.7446219977064, 44.7403307309793),
   (-72.7446535971851, 44.7401021305099),
   (-72.7445251308295, 44.7399651304873),
   (-72.7436267971383, 44.7397831301882),
   (-72.7433055979743, 44.7395091301432),
   (-72.7433689974806, 44.7390975967776),
   (-72.7437521977051, 44.7382515307845),
   (-72.7436233311511, 44.7378859968422),
   (-72.7427889977134, 44.7377039974425),
   (-72.7416669972334, 44.7379337301584),
   (-72.7404807312001, 44.7380719307759),
   (-72.7400955308834, 44.7378665301172),
   (-72.7398703307505, 44.7374553304001),
   (-72.7397415307464, 44.7371353974815),
   (-72.7399011972818, 44.736883

In [19]:
waters._epa_trace_resp_to_esri_geom(trace_response)

{'paths': [[(-72.7413852715841, 44.743546734191),
   (-72.7419019307288, 44.7435333301),
   (-72.7424789978068, 44.7433955305802),
   (-72.7440173304348, 44.7427541304996),
   (-72.74481839795, 44.7423191302246),
   (-72.7445611972408, 44.7419765307932),
   (-72.744592731069, 44.7416793309373),
   (-72.7441423973531, 44.7410625974618),
   (-72.7441415978558, 44.7406283975834),
   (-72.7446219977064, 44.7403307309793),
   (-72.7446535971851, 44.7401021305099),
   (-72.7445251308295, 44.7399651304873),
   (-72.7436267971383, 44.7397831301882),
   (-72.7433055979743, 44.7395091301432),
   (-72.7433689974806, 44.7390975967776),
   (-72.7437521977051, 44.7382515307845),
   (-72.7436233311511, 44.7378859968422),
   (-72.7427889977134, 44.7377039974425),
   (-72.7416669972334, 44.7379337301584),
   (-72.7404807312001, 44.7380719307759),
   (-72.7400955308834, 44.7378665301172),
   (-72.7398703307505, 44.7374553304001),
   (-72.7397415307464, 44.7371353974815),
   (-72.7399011972818, 44.736883

In [39]:
from shapely.geometry import shape

flowline_list = [shape(flowline['shape']) for flowline in resp_json['output']['flowlines_traversed']]
flowline = ops.linemerge(flowline_list)
hydroline = Geometry.from_shapely(flowline, {'wkid': 4269})
hydroline.split_at_point(epa_putin['geometry'])

[{'paths': [[(-72.7413852715841, 44.743546734191),
    (-72.7419019307288, 44.7435333301),
    (-72.7424789978068, 44.7433955305802),
    (-72.7440173304348, 44.7427541304996),
    (-72.74481839795, 44.7423191302246),
    (-72.7445611972408, 44.7419765307932),
    (-72.744592731069, 44.7416793309373),
    (-72.7441423973531, 44.7410625974618),
    (-72.7441415978558, 44.7406283975834),
    (-72.7446219977064, 44.7403307309793),
    (-72.7446535971851, 44.7401021305099),
    (-72.7445251308295, 44.7399651304873),
    (-72.7436267971383, 44.7397831301882),
    (-72.7433055979743, 44.7395091301432),
    (-72.7433689974806, 44.7390975967776),
    (-72.7437521977051, 44.7382515307845),
    (-72.7436233311511, 44.7378859968422),
    (-72.7427889977134, 44.7377039974425),
    (-72.7416669972334, 44.7379337301584),
    (-72.7404807312001, 44.7380719307759),
    (-72.7400955308834, 44.7378665301172),
    (-72.7398703307505, 44.7374553304001),
    (-72.7397415307464, 44.7371353974815),
    (-72.